In [9]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [10]:
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

In [11]:
# Initializing 1st time for a dataset
# nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID)

# When Initializing for first time , you will need to create metadata_json
# nl2sqlbq_client.create_metadata_json(metadata_json_dest_path = "../nl2sql_src/cache_metadata/metadata_cache.json",data_dict_path="../nl2sql_src/data_dict/data_dictionary.json",col_values_distribution = True) #data_dict_path is optional . It is used to provide descriptions for tables and columns

In [12]:
# Initializing when metadata cache is already created
nl2sqlbq_client = Nl2sqlBq(project_id=PROJECT_ID, dataset_id=DATASET_ID, metadata_json_path = "../nl2sql_src/cache_metadata/metadata_cache.json",model_name="text-bison@002", tuned_model=False)

text-bison@002
text-bison@002


In [13]:
# question = "What is the total revenue for the construction primary industry ?"
question = "What are the top 5 industries in terms of revenue?"

In [14]:
# Convert to sql
# table_name = nl2sqlbq_client.table_filter(question) 
query = nl2sqlbq_client.generate_sql(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
print(query)

 
SELECT 
  ZI_ES_INDUSTRY AS industry,
  SUM(ZI_C_REVENUE) AS total_revenue
FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california`
GROUP BY 
  industry
ORDER BY 
  total_revenue DESC
LIMIT 5;



In [15]:
# Execute query
nl2sqlbq_client.execute_query(query)

,industry,total_revenue
0,Retail,281964937
1,Consumer Electronics,86153757
2,Automotive,38272006
3,Apparel & Fashion,16427778
4,Consumer Goods,5971238


In [16]:
#Convert to sql and excute
nl2sqlbq_client.text_to_sql_execute(question)

 
SELECT 
  ZI_ES_INDUSTRY AS industry,
  SUM(ZI_C_REVENUE) AS total_revenue
FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california`
GROUP BY 
  industry
ORDER BY 
  total_revenue DESC
LIMIT 5;



,industry,total_revenue
0,Retail,281964937
1,Consumer Electronics,86153757
2,Automotive,38272006
3,Apparel & Fashion,16427778
4,Consumer Goods,5971238


In [17]:
# If you already know table_name provide it, for more accurate and faster execution by avoiding table_filtering when there are more than 1 table in dataset
sql_execution_response = nl2sqlbq_client.text_to_sql_execute(question, table_name="zi-retail-companies-headquartered-california")
print(sql_execution_response)

 
SELECT 
  ZI_ES_INDUSTRY AS industry,
  SUM(ZI_C_REVENUE) AS total_revenue
FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california`
GROUP BY 
  industry
ORDER BY 
  total_revenue DESC
LIMIT 5;

               industry  total_revenue
0                Retail      281964937
1  Consumer Electronics       86153757
2            Automotive       38272006
3     Apparel & Fashion       16427778
4        Consumer Goods        5971238


In [18]:
# To convert reponse in form of natural language use result2nl
nl2sqlbq_client.result2nl(result = sql_execution_response, question = question)

' The retail industry reigns supreme, generating a whopping $281,964,937 in revenue, far surpassing its competitors. Consumer electronics comes in second with a substantial revenue of $86,153,757, while the automotive industry takes the third spot with a revenue of $38,272,006. Apparel and fashion, as well as consumer goods, follow closely behind, contributing $16,427,778 and $5,971,238 respectively to the total revenue.'

In [19]:
# For testing batch of questions with ground_truth sql
# nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv")

In [20]:
# For testing batch of questions with full functionality
# nl2sqlbq_client.batch_run(test_file_name = "/test_file-1.csv", output_file_name="/test-file-1-output.csv",execute_query = True, result2nl=True, logger_file = "log_batch.txt" )

## Few Shot Prompting

In [21]:
PGPROJ = "sl-test-project-353312"
PGLOCATION = 'us-central1'
PGINSTANCE = "test-nl2sql"
PGDB = "test-db"
PGTABLE = 'myqueries'
PGUSER = "postgres"
PGPWD = "test-nl2sql"
nl2sqlbq_client.init_pgdb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)

PGSQL - class Initiated


In [22]:
query = nl2sqlbq_client.generate_sql_few_shot(question) # Optionally provide table name for faster exection by avoiding table filtering when there are more than 1 table in dataset
print(query)

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata
 
SELECT ZI_C_INDUSTRY_PRIMARY, SUM(ZI_C_REVENUE) AS total_revenue
FROM sl-test-project-363109.zoominfo.`zi-retail-companies-headquartered-california`
GROUP BY ZI_C_INDUSTRY_PRIMARY
ORDER BY total_revenue DESC
LIMIT 5;



#### Verify if the query is valid by performing a dry run on BQ

In [23]:
# Execute query - dry run only
query_status, message = nl2sqlbq_client.execute_query(query, dry_run=True)
print(message, query_status)

Query is valid
Query is valid True


#### Self-reflection if the query is invalid

In [24]:
# Regenerate the query checking for validity. Max-tries = 5
query_status = False
if not query_status:
    query_status, query = nl2sqlbq_client.self_reflection(question=question, query=query) 

Query is valid


In [25]:
result = nl2sqlbq_client.execute_query(query)
nl2sqlbq_client.result2nl(result=result, question=question)

' The retail industry reigns supreme, generating a whopping $314,482,440 in revenue, leaving its competitors in the dust. Manufacturing follows at a distant second with $121,604,063, while Construction, Business Services, and Agriculture bring in $5,179,104, $4,464,523, and $1,672,833 respectively.'